- Normalize
    1. PCK
    2. Body length

- Vector 비교법
    1. Json 그림 그리기
    2. vector 뽑고
    3. L2 norm 비교
    4. cosine similarity 비교
    5. flow 비교

In [17]:
import json
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import natsort
from glob import glob

# X, Y , Z

In [2]:
id = 'yt_newjeans_ditto1'
LABEL_PATH = 'sample/dancingLabel/'+id
LABEL_LIST=natsort.natsorted(glob(LABEL_PATH+'/*.json'))

vector_list=[]
cross_list=[]
for file in LABEL_LIST:
    f = open(file)
    data = json.load(f)
    keypoint = [data[key][:3] for key in data if key != 'frame_time' and key != 'frame_num' ]
    vec_comb=[]
    for i,point in enumerate(keypoint):
        for k in range(i+1,len(keypoint)):
            cross = np.cross(point,keypoint[k])
            vec_comb.append(cross)
    
    cross_list.append(vec_comb)
            
np.array(cross_list).shape
        

(5315, 78, 3)

In [3]:
id = 'yt_newjeans_ditto2'
LABEL_PATH = 'sample/dancingLabel/'+id
LABEL_LIST=natsort.natsorted(glob(LABEL_PATH+'/*.json'))

for file in LABEL_LIST:
    f = open(file)
    data = json.load(f)
    keypoint = [data[key][:3] for key in data if key != 'frame_time' and key != 'frame_num' ]
    vec_comb=[]
    for i,point in enumerate(keypoint):
        for k in range(i+1,len(keypoint)):
            cross = np.cross(point,keypoint[k])
            vec_comb.append(cross)
    
    cross_list.append(vec_comb)
            
np.array(cross_list).shape

(10836, 78, 3)

In [12]:
# reshape input into [samples, timesteps, features]

cross_list=cross_list.swapaxes(1,2)
print(cross_list.shape)
samples=len(cross_list)
steps= 3#13C2
feature=78

print(samples,steps,feature)

(10836, 3, 78)
10836 3 78


In [19]:
from keras import Model ,models, layers, optimizers, utils
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor="val_loss",patience=5)

#define model
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(steps, feature)))
model.add(layers.RepeatVector(steps))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(feature)))

model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(cross_list, cross_list, epochs=200,validation_split=0.2)

# predict
yhat = model.predict(cross_list[0])
yhat

Epoch 1/200
271/271 [==============================] - 10s 18ms/step - loss: 0.0061 - val_loss: 0.0020
Epoch 2/200
271/271 [==============================] - 3s 12ms/step - loss: 0.0023 - val_loss: 7.9220e-04
Epoch 3/200
271/271 [==============================] - 3s 11ms/step - loss: 7.6789e-04 - val_loss: 3.8755e-04
Epoch 4/200
271/271 [==============================] - 3s 12ms/step - loss: 4.2782e-04 - val_loss: 3.0018e-04
Epoch 5/200
271/271 [==============================] - 3s 11ms/step - loss: 3.3493e-04 - val_loss: 2.3335e-04
Epoch 6/200
271/271 [==============================] - 3s 11ms/step - loss: 2.5995e-04 - val_loss: 1.7701e-04
Epoch 7/200
271/271 [==============================] - 3s 11ms/step - loss: 2.0503e-04 - val_loss: 1.3909e-04
Epoch 8/200
271/271 [==============================] - 3s 12ms/step - loss: 1.6227e-04 - val_loss: 1.1419e-04
Epoch 9/200
271/271 [==============================] - 5s 18ms/step - loss: 1.4270e-04 - val_loss: 1.0927e-04
Epoch 10/200
271/271 

ValueError: in user code:

    File "c:\Users\Playdata\Documents\openpose\dancingGame_project\venv\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Playdata\Documents\openpose\dancingGame_project\venv\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Playdata\Documents\openpose\dancingGame_project\venv\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Playdata\Documents\openpose\dancingGame_project\venv\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\Playdata\Documents\openpose\dancingGame_project\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filter

In [28]:
cross_list[0].reshape(1,3,78).shape

(1, 3, 78)

In [31]:
# predict
yhat1000 = model.predict(cross_list[1000].reshape(1,3,78))
yhat1001 = model.predict(cross_list[1001].reshape(1,3,78))
yhat3000 = model.predict(cross_list[3000].reshape(1,3,78))

1/1 [==============================] - 0s 35ms/step


In [36]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

print(cosine_similarity(yhat1000.flatten(),yhat1001.flatten()))
print(cosine_similarity(yhat3000.flatten(),yhat1000.flatten()))
print(cosine_similarity(yhat3000.flatten(),yhat1001.flatten()))

0.99860823
0.83050436
0.8245591


In [37]:
# predict
yhat256 = model.predict(cross_list[256].reshape(1,3,78))
yhat5781 = model.predict(cross_list[5781].reshape(1,3,78))

print(cosine_similarity(yhat256.flatten(),yhat5781.flatten()))
print(cosine_similarity(yhat3000.flatten(),yhat256.flatten()))
print(cosine_similarity(yhat3000.flatten(),yhat5781.flatten()))

1/1 [==============================] - 0s 64ms/step
0.8486301
0.7814016
0.741056


In [20]:
model.save('save_model_200.h5')

In [38]:
from keras import Model ,models, layers, optimizers, utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor="val_loss",patience=5)
model_ckpt = ModelCheckpoint('model_ckpt.h5',save_best_only=True)

#define model
model2 = models.Sequential()
model2.add(layers.LSTM(100, activation='relu', input_shape=(steps, feature)))
model2.add(layers.RepeatVector(steps))
model2.add(layers.LSTM(100, activation='relu', return_sequences=True))
model2.add(layers.TimeDistributed(layers.Dense(feature)))

model2.compile(optimizer='adam', loss='mse')

# fit model
history = model2.fit(cross_list, cross_list, epochs=200,validation_split=0.2,callbacks=[earlystopping,model_ckpt])


Epoch 1/200
271/271 [==============================] - 15s 21ms/step - loss: 0.0061 - val_loss: 0.0020
Epoch 2/200
271/271 [==============================] - 4s 16ms/step - loss: 0.0020 - val_loss: 9.2655e-04
Epoch 3/200
271/271 [==============================] - 4s 16ms/step - loss: 8.5068e-04 - val_loss: 3.7387e-04
Epoch 4/200
271/271 [==============================] - 5s 17ms/step - loss: 4.2719e-04 - val_loss: 3.0929e-04
Epoch 5/200
271/271 [==============================] - 7s 24ms/step - loss: 3.4469e-04 - val_loss: 2.5457e-04
Epoch 6/200
271/271 [==============================] - 6s 23ms/step - loss: 2.9436e-04 - val_loss: 2.3170e-04
Epoch 7/200
271/271 [==============================] - 6s 22ms/step - loss: 2.5179e-04 - val_loss: 1.7698e-04
Epoch 8/200
271/271 [==============================] - 5s 18ms/step - loss: 1.8897e-04 - val_loss: 1.2768e-04
Epoch 9/200
271/271 [==============================] - 4s 16ms/step - loss: 1.4765e-04 - val_loss: 1.1756e-04
Epoch 10/200
271/271 

In [40]:
# predict
yhat256 = model2.predict(cross_list[256].reshape(1,3,78))
yhat5781 = model2.predict(cross_list[5781].reshape(1,3,78))

print(cosine_similarity(yhat256.flatten(),yhat5781.flatten()))
print(cosine_similarity(yhat3000.flatten(),yhat256.flatten()))
print(cosine_similarity(yhat3000.flatten(),yhat5781.flatten()))

1/1 [==============================] - 0s 67ms/step
0.8488801
0.78357387
0.74191666


# X,Y

In [43]:
id = 'yt_newjeans_ditto1'
LABEL_PATH = 'sample/dancingLabel/'+id
LABEL_LIST=natsort.natsorted(glob(LABEL_PATH+'/*.json'))

cross_list=[]

for file in LABEL_LIST:
    f = open(file)
    data = json.load(f)
    keypoint = [data[key][:2] for key in data if key != 'frame_time' and key != 'frame_num' ]
    vec_comb=[]
    for i,point in enumerate(keypoint):
        for k in range(i+1,len(keypoint)):
            # cross = np.cross(point,keypoint[k])
            st_point=point 
            end_point=keypoint[k]
            vec=[ep-sp for sp, ep in zip(st_point, end_point)]
            vec=vec/np.linalg.norm(vec)
            vec_comb.append(vec)
    
    cross_list.append(vec_comb)
            
id = 'yt_newjeans_ditto2'
LABEL_PATH = 'sample/dancingLabel/'+id
LABEL_LIST=natsort.natsorted(glob(LABEL_PATH+'/*.json'))

for file in LABEL_LIST:
    f = open(file)
    data = json.load(f)
    keypoint = [data[key][:2] for key in data if key != 'frame_time' and key != 'frame_num' ]
    vec_comb=[]
    for i,point in enumerate(keypoint):
        for k in range(i+1,len(keypoint)):
            st_point=point 
            end_point=keypoint[k]
            vec=[ep-sp for sp, ep in zip(st_point, end_point)]
            vec=vec/np.linalg.norm(vec)
            vec_comb.append(vec)
    
    cross_list.append(vec_comb)
            
np.array(cross_list).shape
        

(10836, 78, 2)

In [46]:
cross_list=np.array(cross_list).swapaxes(1,2)
print(cross_list.shape)
samples=len(cross_list)
steps= 2
feature=78

print(samples,steps,feature)

(10836, 2, 78)
10836 2 78


In [47]:
from keras import Model ,models, layers, optimizers, utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor="val_loss",patience=5)
model_ckpt = ModelCheckpoint('model_xy_motion_ckpt.h5',save_best_only=True)

#define model
model3 = models.Sequential()
model3.add(layers.LSTM(100, activation='relu', input_shape=(steps, feature)))
model3.add(layers.RepeatVector(steps))
model3.add(layers.LSTM(100, activation='relu', return_sequences=True))
model3.add(layers.TimeDistributed(layers.Dense(feature)))

model3.compile(optimizer='adam', loss='mse')

# fit model
history = model3.fit(cross_list, cross_list, epochs=200,validation_split=0.2,callbacks=[earlystopping,model_ckpt])

Epoch 1/200
271/271 [==============================] - 8s 13ms/step - loss: 0.0909 - val_loss: 0.0232
Epoch 2/200
271/271 [==============================] - 3s 11ms/step - loss: 0.0154 - val_loss: 0.0123
Epoch 3/200
271/271 [==============================] - 2s 9ms/step - loss: 0.0096 - val_loss: 0.0092
Epoch 4/200
271/271 [==============================] - 3s 12ms/step - loss: 0.0074 - val_loss: 0.0075
Epoch 5/200
271/271 [==============================] - 3s 11ms/step - loss: 0.0060 - val_loss: 0.0065
Epoch 6/200
271/271 [==============================] - 3s 10ms/step - loss: 0.0050 - val_loss: 0.0055
Epoch 7/200
271/271 [==============================] - 3s 11ms/step - loss: 0.0044 - val_loss: 0.0049
Epoch 8/200
271/271 [==============================] - 3s 10ms/step - loss: 0.0040 - val_loss: 0.0046
Epoch 9/200
271/271 [==============================] - 3s 12ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 10/200
271/271 [==============================] - 3s 11ms/step - loss: 0.0032

# X,Y,Z - motion

In [48]:
id = 'yt_newjeans_ditto1'
LABEL_PATH = 'sample/dancingLabel/'+id
LABEL_LIST=natsort.natsorted(glob(LABEL_PATH+'/*.json'))

cross_list=[]

for file in LABEL_LIST:
    f = open(file)
    data = json.load(f)
    keypoint = [data[key][:3] for key in data if key != 'frame_time' and key != 'frame_num' ]
    vec_comb=[]
    for i,point in enumerate(keypoint):
        for k in range(i+1,len(keypoint)):
            # cross = np.cross(point,keypoint[k])
            st_point=point 
            end_point=keypoint[k]
            vec=[ep-sp for sp, ep in zip(st_point, end_point)]
            vec=vec/np.linalg.norm(vec)
            vec_comb.append(vec)
    
    cross_list.append(vec_comb)
            
id = 'yt_newjeans_ditto2'
LABEL_PATH = 'sample/dancingLabel/'+id
LABEL_LIST=natsort.natsorted(glob(LABEL_PATH+'/*.json'))

for file in LABEL_LIST:
    f = open(file)
    data = json.load(f)
    keypoint = [data[key][:3] for key in data if key != 'frame_time' and key != 'frame_num' ]
    vec_comb=[]
    for i,point in enumerate(keypoint):
        for k in range(i+1,len(keypoint)):
            st_point=point 
            end_point=keypoint[k]
            vec=[ep-sp for sp, ep in zip(st_point, end_point)]
            vec=vec/np.linalg.norm(vec)
            vec_comb.append(vec)
    
    cross_list.append(vec_comb)
            
cross_list=np.array(cross_list).swapaxes(1,2)
print(cross_list.shape)
samples=len(cross_list)
steps= 3
feature=78
        

(10836, 3, 78)


In [50]:
from keras import Model ,models, layers, optimizers, utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor="val_loss",patience=5)
model_ckpt = ModelCheckpoint('model_xyz_motion_ckpt.h5',save_best_only=True)

#define model
model4 = models.Sequential()
model4.add(layers.LSTM(100, activation='relu', input_shape=(steps, feature)))
model4.add(layers.RepeatVector(steps))
model4.add(layers.LSTM(100, activation='relu', return_sequences=True))
model4.add(layers.TimeDistributed(layers.Dense(feature)))

model4.compile(optimizer='adam', loss='mse')

history = model4.fit(cross_list, cross_list, epochs=200,validation_split=0.2,callbacks=[earlystopping,model_ckpt])

# fit model

Epoch 1/200
271/271 [==============================] - 9s 16ms/step - loss: 0.0894 - val_loss: 0.0266
Epoch 2/200
271/271 [==============================] - 4s 14ms/step - loss: 0.0204 - val_loss: 0.0165
Epoch 3/200
271/271 [==============================] - 3s 12ms/step - loss: 0.0138 - val_loss: 0.0121
Epoch 4/200
271/271 [==============================] - 4s 15ms/step - loss: 0.0106 - val_loss: 0.0097
Epoch 5/200
271/271 [==============================] - 4s 14ms/step - loss: 0.0090 - val_loss: 0.0082
Epoch 6/200
271/271 [==============================] - 4s 13ms/step - loss: 0.0080 - val_loss: 0.0078
Epoch 7/200
271/271 [==============================] - 4s 14ms/step - loss: 0.0073 - val_loss: 0.0070
Epoch 8/200
271/271 [==============================] - 3s 13ms/step - loss: 0.0067 - val_loss: 0.0067
Epoch 9/200
271/271 [==============================] - 4s 14ms/step - loss: 0.0062 - val_loss: 0.0062
Epoch 10/200
271/271 [==============================] - 4s 14ms/step - loss: 0.005